# Score Functions & SDEs

**Module 7.2, Lesson 1** | CourseAI

You know the theory—the score function is the gradient of log probability, it points toward regions of higher probability, and DDPM's noise prediction is a scaled version of the score. This notebook makes that concrete. You will compute score functions by hand, visualize them as vector fields, verify the score-noise equivalence with a real model, and compare SDE vs ODE sampling trajectories.

**What you will do:**
- Compute score functions analytically for 1D Gaussians and a Gaussian mixture, plot them alongside the PDF
- Visualize the score as a 2D vector field for a Gaussian mixture and observe how noise smooths the field
- Verify the score-noise equivalence using a pre-trained diffusion model on a toy 2D distribution
- Compare SDE (stochastic, wiggly) vs ODE (deterministic, smooth) sampling trajectories

**For each exercise, PREDICT the output before running the cell.**

Every concept in this notebook comes from the lesson. Score function as compass toward data, the score-noise equivalence, the reverse SDE vs probability flow ODE. No new theory—just hands-on practice with the math and models.

**Estimated time:** 30–45 minutes. Exercises 1–2 are pure math (no GPU needed). Exercises 3–4 train a small model on a toy 2D distribution (~1 minute on CPU).

## Setup

Run this cell to install dependencies, import everything, and configure the environment.

No GPU required for this notebook. Everything runs on CPU. The diffusion model in Exercises 3–4 is a tiny MLP trained on a 2D Gaussian mixture—not an image model.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy.stats import norm

# Reproducible results
torch.manual_seed(42)
np.random.seed(42)

# Nice plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [10, 4]
plt.rcParams['figure.dpi'] = 100

print('Setup complete. No GPU needed for this notebook.')

## Shared Helpers

A toy diffusion model for Exercises 3–4. This trains a small MLP to predict noise on a 2D Gaussian mixture—the same DDPM training objective you already know, just on 2D points instead of images.

Run this cell now. It defines the model architecture, the noise schedule, and a training function. The actual training happens in Exercise 3.

In [ ]:
# ============================================================
# Toy 2D diffusion model
# ============================================================
# This is DDPM on 2D points instead of images. Same concepts:
#   - Forward process: x_t = sqrt(alpha_bar_t) * x_0 + sqrt(1 - alpha_bar_t) * epsilon
#   - Training: predict epsilon from (x_t, t)
#   - The model is a small MLP, not a U-Net (2D points don't need convolutions)

def make_noise_schedule(T=100, beta_start=1e-4, beta_end=0.02):
    """Linear noise schedule, same as DDPM."""
    betas = torch.linspace(beta_start, beta_end, T)
    alphas = 1.0 - betas
    alpha_bars = torch.cumprod(alphas, dim=0)
    return betas, alphas, alpha_bars


class ToyDiffusionModel(nn.Module):
    """MLP that predicts noise epsilon given (x_t, t).
    
    Input: 2D point x_t concatenated with timestep embedding.
    Output: 2D noise prediction epsilon_theta.
    """
    def __init__(self, hidden_dim=128, T=100):
        super().__init__()
        self.T = T
        # Timestep embedding: map integer t to a learned vector
        self.time_embed = nn.Sequential(
            nn.Embedding(T, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
        )
        # Main network: takes (x_t, time_embedding) -> epsilon prediction
        self.net = nn.Sequential(
            nn.Linear(2 + hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 2),  # Output: 2D noise prediction
        )

    def forward(self, x_t, t):
        """Predict the noise that was added to create x_t at timestep t."""
        t_emb = self.time_embed(t)
        inp = torch.cat([x_t, t_emb], dim=-1)
        return self.net(inp)


def sample_gaussian_mixture(n, centers=None, std=0.3):
    """Sample from a 2D Gaussian mixture.
    
    Default: 4 clusters arranged in a square pattern.
    This is the "data distribution" our toy model learns.
    """
    if centers is None:
        centers = torch.tensor([
            [-2.0, -2.0],
            [-2.0,  2.0],
            [ 2.0, -2.0],
            [ 2.0,  2.0],
        ])
    k = centers.shape[0]
    # Randomly assign each point to a cluster
    assignments = torch.randint(0, k, (n,))
    samples = centers[assignments] + std * torch.randn(n, 2)
    return samples


def train_toy_diffusion(model, alpha_bars, n_epochs=200, batch_size=512, lr=3e-4):
    """Train the toy diffusion model with the DDPM objective.
    
    Same training loop as DDPM on images:
    1. Sample clean data x_0
    2. Sample random timestep t and noise epsilon
    3. Create noisy x_t = sqrt(alpha_bar_t) * x_0 + sqrt(1 - alpha_bar_t) * epsilon
    4. Predict epsilon from (x_t, t)
    5. Loss = MSE(epsilon, epsilon_predicted)
    """
    T = len(alpha_bars)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    losses = []
    for epoch in range(n_epochs):
        # 1. Sample clean data
        x_0 = sample_gaussian_mixture(batch_size)
        
        # 2. Sample random timesteps and noise
        t = torch.randint(0, T, (batch_size,))
        epsilon = torch.randn_like(x_0)
        
        # 3. Create noisy samples: x_t = sqrt(alpha_bar_t) * x_0 + sqrt(1 - alpha_bar_t) * epsilon
        ab_t = alpha_bars[t].unsqueeze(-1)  # (batch, 1)
        x_t = torch.sqrt(ab_t) * x_0 + torch.sqrt(1 - ab_t) * epsilon
        
        # 4. Predict noise
        epsilon_pred = model(x_t, t)
        
        # 5. MSE loss
        loss = nn.functional.mse_loss(epsilon_pred, epsilon)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        if (epoch + 1) % 50 == 0:
            print(f'  Epoch {epoch+1}/{n_epochs}, loss: {loss.item():.4f}')
    
    return losses


print('Toy diffusion model defined.')
print('Architecture: MLP with timestep embedding (same DDPM objective, just on 2D points).')

---

## Exercise 1: Compute Score Functions by Hand `[Guided]`

From the lesson: the score function is $\text{score}(x) = \nabla_x \log p(x)$. It is a compass toward likely data—it points in the direction of increasing probability. For a standard Gaussian $\mathcal{N}(0,1)$, the score is simply $-x$.

We will compute the score function analytically for three distributions:
1. $\mathcal{N}(0, 1)$—the simplest case
2. $\mathcal{N}(3, 2)$—shifted and wider
3. A mixture of two Gaussians—two peaks, where things get interesting

For each, we plot the PDF and the score function side by side. The score should be zero at peaks (you are already at high probability—nowhere better to go) and large far from peaks (strong pull back toward the data).

**Before running, predict:**
- For the Gaussian mixture with peaks at $\mu_1 = -2$ and $\mu_2 = 3$, where will the score cross zero? How many zero crossings will there be?
- Between the two peaks, which direction will the score point?

In [ ]:
# ============================================================
# Exercise 1: Compute score functions analytically
# ============================================================

x = np.linspace(-8, 8, 1000)

# --- Distribution 1: N(0, 1) ---
# log p(x) = -x^2/2 + const
# score(x) = d/dx log p(x) = -x
pdf_1 = norm.pdf(x, loc=0, scale=1)
score_1 = -x  # Analytic score for N(0,1)

# --- Distribution 2: N(3, 2) ---
# For N(mu, sigma^2): log p(x) = -(x-mu)^2 / (2*sigma^2) + const
# score(x) = -(x - mu) / sigma^2
mu_2, sigma_2 = 3.0, 2.0
pdf_2 = norm.pdf(x, loc=mu_2, scale=sigma_2)
score_2 = -(x - mu_2) / (sigma_2 ** 2)  # Analytic score for N(3, 2)

# --- Distribution 3: Mixture of two Gaussians ---
# p(x) = 0.5 * N(x; -2, 0.8) + 0.5 * N(x; 3, 1.0)
# score(x) = (d/dx p(x)) / p(x)  (because d/dx log p = (d/dx p) / p)
mu_a, sigma_a = -2.0, 0.8
mu_b, sigma_b = 3.0, 1.0
weight_a, weight_b = 0.5, 0.5

pdf_a = norm.pdf(x, loc=mu_a, scale=sigma_a)
pdf_b = norm.pdf(x, loc=mu_b, scale=sigma_b)
pdf_3 = weight_a * pdf_a + weight_b * pdf_b

# Derivative of each Gaussian component:
# d/dx N(x; mu, sigma) = -(x - mu) / sigma^2 * N(x; mu, sigma)
dpdf_a = -(x - mu_a) / (sigma_a ** 2) * pdf_a
dpdf_b = -(x - mu_b) / (sigma_b ** 2) * pdf_b
dpdf_3 = weight_a * dpdf_a + weight_b * dpdf_b

# Score = (d/dx p(x)) / p(x)
score_3 = dpdf_3 / (pdf_3 + 1e-10)  # Small epsilon to avoid division by zero

print('Score functions computed analytically.')
print('')
print('Key formulas:')
print('  N(0, 1):      score(x) = -x')
print('  N(mu, sig^2): score(x) = -(x - mu) / sig^2')
print('  Mixture:      score(x) = (d/dx p(x)) / p(x)')

In [ ]:
# Plot PDF and score function side by side for all three distributions

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

distributions = [
    ('$\\mathcal{N}(0, 1)$', pdf_1, score_1, [0]),
    ('$\\mathcal{N}(3, 2)$', pdf_2, score_2, [3]),
    ('Mixture: $0.5\\,\\mathcal{N}(-2, 0.8) + 0.5\\,\\mathcal{N}(3, 1.0)$', pdf_3, score_3, [-2, 3]),
]

for i, (title, pdf, score, peaks) in enumerate(distributions):
    # Left column: PDF
    axes[i, 0].plot(x, pdf, color='#60a5fa', linewidth=2)
    axes[i, 0].fill_between(x, pdf, alpha=0.2, color='#60a5fa')
    for p in peaks:
        axes[i, 0].axvline(p, color='#f97316', linestyle='--', alpha=0.6, label=f'peak at x={p}')
    axes[i, 0].set_title(f'{title}\nProbability Density', fontsize=11)
    axes[i, 0].set_xlabel('x')
    axes[i, 0].set_ylabel('p(x)')
    axes[i, 0].legend(fontsize=8)
    
    # Right column: Score function
    axes[i, 1].plot(x, score, color='#a78bfa', linewidth=2)
    axes[i, 1].axhline(0, color='white', linewidth=0.5, alpha=0.3)
    for p in peaks:
        axes[i, 1].axvline(p, color='#f97316', linestyle='--', alpha=0.6)
    # Mark zero crossings
    zero_crossings = np.where(np.diff(np.sign(score)))[0]
    for zc in zero_crossings:
        axes[i, 1].plot(x[zc], 0, 'o', color='#f97316', markersize=8)
    axes[i, 1].set_title(f'{title}\nScore Function $\\nabla_x \\log p(x)$', fontsize=11)
    axes[i, 1].set_xlabel('x')
    axes[i, 1].set_ylabel('score(x)')
    axes[i, 1].set_ylim(-6, 6)

plt.tight_layout()
plt.show()

print('Observations:')
print('- N(0,1): Score is a straight line through the origin. score(x) = -x.')
print('  At x=3, score=-3 (strong pull left). At x=0, score=0 (at the peak).')
print('')
print('- N(3,2): Score is a gentler line through x=3. score(x) = -(x-3)/4.')
print('  The wider variance makes the score shallower\u2014the probability landscape is flatter.')
print('')
print('- Mixture: THREE zero crossings! One at each peak (score=0 at the top of each')
print('  Gaussian), and one BETWEEN the peaks (a saddle point where the pulls balance).')
print('  Between the peaks, the score points toward the NEAREST peak.')
print('  Far from both peaks, the score points toward the nearest one.')

### What Just Happened

You computed the score function analytically for three distributions and verified the lesson's core claims:

- **The score is zero at peaks.** At the mean of each Gaussian, the score crosses zero. This is the same as gradient = 0 at a minimum in optimization. Zero score means "you are at a peak—no better direction to go."
- **The score is large far from peaks.** Points far from the data experience a strong pull back toward high-probability regions. The further away, the stronger the pull.
- **For the mixture, the score is more interesting.** There are three zero crossings: one at each peak, and one between them where the opposing pulls balance. Between the peaks, the score points toward the nearest peak—it is a compass toward the closest mode of the distribution.
- **Wider variance = shallower score.** The $\mathcal{N}(3, 2)$ score is gentler than the $\mathcal{N}(0, 1)$ score because the probability landscape is flatter. The score magnitude reflects how steeply probability changes.

The score function is a direction at every point—not a value. It says "which way is up," not "how high am I."

---

## Exercise 2: Visualize the Score as a 2D Vector Field `[Guided]`

From the lesson: in 2D, the score at every point is a 2D vector. For a Gaussian mixture with multiple peaks, the score field shows arrows converging on each peak. At different noise levels, the field changes—high noise simplifies the field (nearly Gaussian, nearly linear arrows), low noise reveals the complex structure (distinct basins of attraction).

We will:
1. Create a 2D Gaussian mixture with two peaks
2. Plot the score field as a quiver plot (arrows at every point)
3. Add increasing Gaussian noise and re-plot the score field at each noise level

**Before running, predict:**
- At very high noise ($\sigma = 3.0$), will the score field show two distinct basins, or will it look like a single Gaussian?
- At very low noise ($\sigma = 0.1$), will the arrows between the two peaks be simple or complex?
- How does this connect to the noise schedule? (Think about $\bar\alpha$ near 0 vs near 1.)

In [ ]:
# ============================================================
# Exercise 2: Score function as a 2D vector field
# ============================================================

def gaussian_2d_pdf(x, y, mu, cov_inv, det_cov):
    """Compute the PDF of a 2D Gaussian at grid points (x, y)."""
    dx = x - mu[0]
    dy = y - mu[1]
    exponent = -0.5 * (cov_inv[0,0]*dx**2 + 2*cov_inv[0,1]*dx*dy + cov_inv[1,1]*dy**2)
    return (1.0 / (2 * np.pi * np.sqrt(det_cov))) * np.exp(exponent)


def score_2d_gaussian_mixture(x, y, means, covariances, weights):
    """Compute the score field for a 2D Gaussian mixture.
    
    score(x) = nabla_x log p(x) = (nabla_x p(x)) / p(x)
    
    For a mixture, nabla_x p(x) = sum_k w_k * nabla_x N(x; mu_k, Sigma_k)
    and nabla_x N(x; mu_k, Sigma_k) = -Sigma_k^{-1} (x - mu_k) * N(x; mu_k, Sigma_k)
    """
    p_total = np.zeros_like(x)
    grad_x_total = np.zeros_like(x)
    grad_y_total = np.zeros_like(x)
    
    for mu, cov, w in zip(means, covariances, weights):
        cov_inv = np.linalg.inv(cov)
        det_cov = np.linalg.det(cov)
        
        pdf = gaussian_2d_pdf(x, y, mu, cov_inv, det_cov)
        p_total += w * pdf
        
        # Gradient of the k-th component: -Sigma_k^{-1} (x - mu_k) * N(x; mu_k, Sigma_k)
        dx = x - mu[0]
        dy = y - mu[1]
        grad_x_total += w * pdf * (-(cov_inv[0,0]*dx + cov_inv[0,1]*dy))
        grad_y_total += w * pdf * (-(cov_inv[1,0]*dx + cov_inv[1,1]*dy))
    
    # score = grad(log p) = grad(p) / p
    score_x = grad_x_total / (p_total + 1e-10)
    score_y = grad_y_total / (p_total + 1e-10)
    
    return score_x, score_y, p_total


# Define a 2D Gaussian mixture with two peaks
means = [np.array([-2.0, 0.0]), np.array([2.0, 0.0])]
covariances = [np.array([[0.5, 0.0], [0.0, 0.5]]),
               np.array([[0.5, 0.0], [0.0, 0.5]])]
weights = [0.5, 0.5]

# Create grid for plotting
grid_range = 5.0
n_grid = 20  # Number of arrows per axis (keep sparse for readability)
n_density = 200  # Number of points for density contours

xg = np.linspace(-grid_range, grid_range, n_grid)
yg = np.linspace(-grid_range, grid_range, n_grid)
X, Y = np.meshgrid(xg, yg)

xd = np.linspace(-grid_range, grid_range, n_density)
yd = np.linspace(-grid_range, grid_range, n_density)
XD, YD = np.meshgrid(xd, yd)

print('2D Gaussian mixture defined:')
print(f'  Peak 1: mean = {means[0]}, covariance = {covariances[0][0,0]:.1f} * I')
print(f'  Peak 2: mean = {means[1]}, covariance = {covariances[1][0,0]:.1f} * I')
print(f'  Equal weights: {weights}')

In [ ]:
# Plot the score field at the original (clean) distribution

score_x, score_y, p_total = score_2d_gaussian_mixture(X, Y, means, covariances, weights)
_, _, p_density = score_2d_gaussian_mixture(XD, YD, means, covariances, weights)

# Compute arrow magnitude for coloring
magnitude = np.sqrt(score_x**2 + score_y**2)

fig, ax = plt.subplots(1, 1, figsize=(8, 7))

# Density contours in the background
ax.contourf(XD, YD, p_density, levels=20, cmap='Blues', alpha=0.3)

# Score arrows (quiver plot)
# Normalize arrow lengths for visual clarity
norm_factor = magnitude.max()
quiv = ax.quiver(X, Y, score_x/norm_factor, score_y/norm_factor, magnitude,
                 cmap='cool', scale=25, width=0.004, alpha=0.9)

# Mark the peaks
for mu in means:
    ax.plot(mu[0], mu[1], '*', color='#f97316', markersize=15, markeredgecolor='white', markeredgewidth=1)

ax.set_title('Score Field of a 2D Gaussian Mixture (Clean Distribution)', fontsize=13)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_aspect('equal')
plt.colorbar(quiv, ax=ax, label='Score magnitude')
plt.tight_layout()
plt.show()

print('The arrows point toward the nearest peak everywhere.')
print('Between the peaks, the field splits\u2014some points are pulled left, others right.')
print('At each peak (orange stars), the arrows converge and the score approaches zero.')

In [ ]:
# Now add increasing noise and observe how the score field changes.
#
# Adding Gaussian noise N(0, sigma^2 * I) to the data distribution is equivalent
# to convolving the distribution with a Gaussian. This smooths the distribution
# and simplifies the score field.
#
# This is exactly what happens in the DDPM forward process at different timesteps.
# High noise (large sigma, alpha_bar near 0) -> simple score field.
# Low noise (small sigma, alpha_bar near 1) -> complex score field.

noise_levels = [0.0, 0.3, 1.0, 3.0]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for idx, sigma_noise in enumerate(noise_levels):
    # Adding noise sigma to each component increases its covariance by sigma^2 * I
    noisy_covariances = [cov + sigma_noise**2 * np.eye(2) for cov in covariances]
    
    sx, sy, _ = score_2d_gaussian_mixture(X, Y, means, noisy_covariances, weights)
    _, _, pd = score_2d_gaussian_mixture(XD, YD, means, noisy_covariances, weights)
    
    mag = np.sqrt(sx**2 + sy**2)
    nf = max(mag.max(), 1e-6)
    
    ax = axes[idx]
    ax.contourf(XD, YD, pd, levels=20, cmap='Blues', alpha=0.3)
    ax.quiver(X, Y, sx/nf, sy/nf, mag, cmap='cool', scale=25, width=0.005, alpha=0.9)
    for mu in means:
        ax.plot(mu[0], mu[1], '*', color='#f97316', markersize=12,
                markeredgecolor='white', markeredgewidth=1)
    
    noise_label = 'Clean' if sigma_noise == 0 else f'$\\sigma$ = {sigma_noise}'
    ax.set_title(f'{noise_label}', fontsize=12)
    ax.set_xlim(-grid_range, grid_range)
    ax.set_ylim(-grid_range, grid_range)
    ax.set_aspect('equal')
    ax.set_xlabel('$x_1$')
    if idx == 0:
        ax.set_ylabel('$x_2$')

plt.suptitle(
    'Score Field at Increasing Noise Levels\n'
    'High noise \u2192 simple field (easy task) | Low noise \u2192 complex field (hard task)',
    fontsize=13, y=1.05
)
plt.tight_layout()
plt.show()

print('Key observations:')
print('')
print('- Clean (sigma=0): Two distinct basins. Sharp boundary between them.')
print('  The score field is complex\u2014the model must capture fine structure.')
print('')
print('- Sigma=0.3: The basins start to merge. The boundary softens.')
print('')
print('- Sigma=1.0: The two peaks are barely distinguishable.')
print('  The score field is becoming simpler\u2014nearly radial.')
print('')
print('- Sigma=3.0: The distribution is effectively a single Gaussian.')
print('  The score field is nearly linear\u2014all arrows point toward the center.')
print('  A model can capture this trivially.')
print('')
print('Connect to alpha-bar: This is why the noise schedule starts with easy tasks')
print('(high noise, alpha_bar near 0, simple scores) and progresses to hard tasks')
print('(low noise, alpha_bar near 1, complex scores). The denoising model learns')
print('the easy landscape first, then gradually handles finer structure.')

### What Just Happened

You visualized the score function as a 2D vector field and observed its behavior at different noise levels:

- **Clean distribution:** Two distinct basins of attraction. The score field is complex—sharp boundaries between regions pulled toward different peaks. A model must capture this fine structure.
- **Low noise ($\sigma = 0.3$):** The basins begin to merge. The boundary between them softens. Still two peaks, but the transition is smoother.
- **Medium noise ($\sigma = 1.0$):** The two peaks are barely distinguishable. The score field is becoming nearly radial—almost like a single Gaussian.
- **High noise ($\sigma = 3.0$):** Effectively a single Gaussian. The score field is nearly linear. All arrows point toward the combined center. Trivial for a model to learn.

This is exactly what the denoising model sees at different timesteps. At high noise ($\bar\alpha$ near 0), the model's job is easy—the noisy distribution is nearly Gaussian, the score field is simple. At low noise ($\bar\alpha$ near 1), the model must capture the fine structure of the data distribution—multiple modes, sharp boundaries, complex geometry. The noise schedule controls the difficulty progression.

---

## Exercise 3: Verify the Score-Noise Equivalence `[Supported]`

From the lesson: the DDPM noise prediction $\epsilon_\theta(x_t, t)$ is related to the score by:

$$\text{score}(x_t, t) \approx -\frac{\epsilon_\theta(x_t, t)}{\sqrt{1 - \bar\alpha_t}}$$

The noise prediction points **away** from data (it estimates the noise that corrupted the sample). The score points **toward** data (it estimates the direction of increasing probability). They are the same direction, opposite sign, with a scaling factor.

We will:
1. Train the toy diffusion model from the Helpers section on our 2D Gaussian mixture
2. For given noisy samples $x_t$ at specific timesteps, get the model's noise prediction
3. Compute the implied score from the noise prediction
4. Verify that the score points from $x_t$ toward the data clusters

Your task: fill in the TODOs to compute the implied score and verify its direction.

**Before running, predict:**
- At a high timestep (lots of noise), will the implied score arrows be large or small?
- At a low timestep (little noise), will the score arrows point more precisely toward the nearest cluster?

In [ ]:
# Step 1: Train the toy diffusion model (~30 seconds)

T = 100  # Number of diffusion timesteps
betas, alphas, alpha_bars = make_noise_schedule(T=T)

model = ToyDiffusionModel(hidden_dim=128, T=T)

print('Training toy diffusion model on 2D Gaussian mixture...')
print('(Same DDPM objective: predict the noise epsilon from noisy samples x_t)')
print()
losses = train_toy_diffusion(model, alpha_bars, n_epochs=300, batch_size=512)

# Plot training loss
fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.plot(losses, color='#60a5fa', linewidth=1, alpha=0.7)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss')
ax.set_title('Training Loss (DDPM on 2D Gaussian Mixture)')
plt.tight_layout()
plt.show()

print(f'\nFinal loss: {losses[-1]:.4f}')
print('The model has learned to predict noise\u2014which means it has implicitly')
print('learned the score function at every noise level.')

In [ ]:
# Step 2: Verify the score-noise equivalence
#
# For several timesteps, we will:
#   a) Create noisy samples x_t from clean data x_0
#   b) Get the model's noise prediction epsilon_theta(x_t, t)
#   c) Compute the implied score: score = -epsilon_theta / sqrt(1 - alpha_bar_t)
#   d) Verify: the score should point from x_t toward the data clusters

model.eval()

timesteps_to_check = [10, 30, 60, 90]  # Low noise to high noise

# Generate some clean data points
torch.manual_seed(123)
x_0 = sample_gaussian_mixture(200)

fig, axes = plt.subplots(1, len(timesteps_to_check), figsize=(20, 5))

for idx, t_val in enumerate(timesteps_to_check):
    t_tensor = torch.full((x_0.shape[0],), t_val, dtype=torch.long)
    
    # Create noisy samples using the forward process
    ab_t = alpha_bars[t_val]
    epsilon = torch.randn_like(x_0)
    x_t = torch.sqrt(ab_t) * x_0 + torch.sqrt(1 - ab_t) * epsilon
    
    # Get model's noise prediction
    with torch.no_grad():
        eps_pred = model(x_t, t_tensor)
    
    # TODO: Compute the implied score from the noise prediction.
    # Remember the formula from the lesson:
    #   score = -epsilon_theta / sqrt(1 - alpha_bar_t)
    #
    # Hint: ab_t is alpha_bar at this timestep. eps_pred is the model's noise prediction.
    # The result should be a tensor with the same shape as eps_pred.
    implied_score = None  # <-- Replace this line
    
    # TODO: Compute the direction from x_t toward the nearest clean data center.
    # This is the "ground truth" direction the score should approximately point.
    #
    # For each noisy point x_t[i], find which cluster center it is closest to,
    # then compute the direction vector (center - x_t[i]).
    #
    # Hint: The cluster centers are at [-2, -2], [-2, 2], [2, -2], [2, 2].
    # Use torch.cdist to find distances, then torch.argmin for closest center.
    centers = torch.tensor([[-2., -2.], [-2., 2.], [2., -2.], [2., 2.]])
    gt_direction = None  # <-- Replace this line
    
    # Plotting (provided\u2014no TODO here)
    ax = axes[idx]
    x_np = x_t.numpy()
    
    # Plot noisy samples
    ax.scatter(x_np[:, 0], x_np[:, 1], s=3, alpha=0.3, color='#94a3b8')
    
    # Plot implied score arrows (subsample for clarity)
    step = 5
    if implied_score is not None:
        score_np = implied_score.numpy()
        # Normalize for visual clarity
        score_mag = np.sqrt(score_np[:, 0]**2 + score_np[:, 1]**2).reshape(-1, 1)
        score_normalized = score_np / (score_mag + 1e-6)
        ax.quiver(x_np[::step, 0], x_np[::step, 1],
                  score_normalized[::step, 0], score_normalized[::step, 1],
                  color='#a78bfa', scale=20, width=0.005, alpha=0.7,
                  label='Implied score')
    
    if gt_direction is not None:
        gt_np = gt_direction.numpy()
        gt_mag = np.sqrt(gt_np[:, 0]**2 + gt_np[:, 1]**2).reshape(-1, 1)
        gt_normalized = gt_np / (gt_mag + 1e-6)
        ax.quiver(x_np[::step, 0], x_np[::step, 1],
                  gt_normalized[::step, 0], gt_normalized[::step, 1],
                  color='#f97316', scale=20, width=0.005, alpha=0.4,
                  label='Toward nearest center')
    
    # Mark cluster centers
    for c in centers.numpy():
        ax.plot(c[0], c[1], '*', color='#f97316', markersize=12,
                markeredgecolor='white', markeredgewidth=1)
    
    ab_val = ab_t.item()
    ax.set_title(f't = {t_val}\n$\\bar\\alpha_t$ = {ab_val:.3f}', fontsize=11)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_aspect('equal')
    if idx == 0:
        ax.legend(fontsize=8, loc='lower left')

plt.suptitle(
    'Score-Noise Equivalence: Implied Score (purple) vs Direction to Nearest Center (orange)\n'
    'Low t = low noise (clustered) | High t = high noise (spread out)',
    fontsize=12, y=1.05
)
plt.tight_layout()
plt.show()

if implied_score is not None:
    print('Observations:')
    print('- The purple (implied score) and orange (ground truth direction) arrows')
    print('  point in similar directions\u2014the score-noise equivalence works!')
    print('')
    print('- At low t (little noise): samples are clustered near the data.')
    print('  The score points precisely toward the nearest cluster center.')
    print('')
    print('- At high t (lots of noise): samples are spread out.')
    print('  The score points generally "inward" toward the data region.')
    print('  Less precise, but still directionally correct.')
else:
    print('implied_score is None\u2014go back and fill in the TODOs.')
    print('Check the solution below if you get stuck.')

<details>
<summary>💡 Solution</summary>

The key insight is that the score-noise equivalence is a direct algebraic relationship. The noise prediction and the score point in opposite directions (noise points away from data, score points toward data), with a scaling factor that depends on the noise level.

```python
# Implied score from noise prediction:
# score = -epsilon_theta / sqrt(1 - alpha_bar_t)
implied_score = -eps_pred / torch.sqrt(1 - ab_t)

# Direction toward nearest center:
# For each point, find the closest cluster center and compute the direction vector.
dists = torch.cdist(x_t, centers)  # (n_points, n_centers)
closest = torch.argmin(dists, dim=1)  # Index of nearest center for each point
gt_direction = centers[closest] - x_t  # Vector from x_t toward nearest center
```

**Why the negative sign:** The model predicts $\epsilon$—the noise that was ADDED to corrupt the data. Noise goes from clean to noisy, so it points AWAY from the data. The score points TOWARD the data. Hence the negative sign: `score = -epsilon / scale`.

**Why the scaling factor:** At high noise ($\bar\alpha_t$ small, $1 - \bar\alpha_t$ large), the scaling factor $1/\sqrt{1 - \bar\alpha_t}$ is close to 1—the noise prediction and score have similar magnitude. At low noise ($\bar\alpha_t$ large, $1 - \bar\alpha_t$ small), the factor becomes large—small noise predictions map to large scores. This makes sense: near-clean data has sharply peaked probability, so the score gradient is steep.

**Common mistake:** Forgetting the negative sign. Without it, the arrows point away from the data instead of toward it.

</details>

### What Just Happened

You verified the lesson's core reveal: the noise prediction IS a scaled version of the score function. Specifically:

- The **implied score** (computed from the model's noise prediction using the equivalence formula) consistently points toward the data clusters. The model was learning the score function all along—it just looked like it was predicting noise.
- At **low noise** (low t, $\bar\alpha$ near 1), the score points precisely toward the nearest cluster. The model captures the fine structure of the data distribution.
- At **high noise** (high t, $\bar\alpha$ near 0), the score points generally inward. The model captures the broad shape but not the fine details.
- The **orange arrows** (ground truth direction to nearest center) and **purple arrows** (implied score) align, confirming the equivalence.

This is the practical proof of $\text{score}(x_t, t) \approx -\epsilon_\theta(x_t, t) / \sqrt{1 - \bar\alpha_t}$. The noise prediction network was a score estimator all along.

---

## Exercise 4: SDE vs ODE Trajectories `[Independent]`

From the lesson: the reverse SDE and probability flow ODE produce the same marginal distributions but follow different paths. The reverse SDE adds stochastic noise at each step (like DDPM sampling)—paths are wiggly and diverse. The probability flow ODE is deterministic (like DDIM sampling)—paths are smooth, and the same starting point always gives the same endpoint.

**Your task:**

Using the trained toy model from Exercise 3, implement both sampling methods:

1. **Reverse SDE sampling** (DDPM-like): At each step, predict noise, remove it, AND add fresh stochastic noise. Different runs from the same starting point should give different endpoints.

2. **Probability flow ODE sampling** (DDIM-like): At each step, predict noise and move deterministically toward clean data. NO stochastic noise injection. Same starting point should always give the same endpoint.

Then:
3. Plot several trajectories from the **same starting points** for both methods.
4. Observe: SDE paths are wiggly and diverse. ODE paths are smooth and deterministic.
5. Run the ODE twice from the same start—verify identical endpoints. Run the SDE twice—verify different endpoints.

**Implementation hints:**

The DDPM reverse step (reverse SDE, discrete approximation):
```
x_{t-1} = (1/sqrt(alpha_t)) * (x_t - (beta_t / sqrt(1 - alpha_bar_t)) * epsilon_theta(x_t, t)) + sigma_t * z
```
where `sigma_t = sqrt(beta_t)` and `z ~ N(0, I)` (set `z = 0` for the last step).

The DDIM step (probability flow ODE, discrete approximation):
```
predicted_x0 = (x_t - sqrt(1 - alpha_bar_t) * epsilon_theta) / sqrt(alpha_bar_t)
x_{t-1} = sqrt(alpha_bar_{t-1}) * predicted_x0 + sqrt(1 - alpha_bar_{t-1}) * epsilon_theta
```

**Before running, predict:**
- If you run the ODE from the same starting noise twice (same seed), will the trajectories overlap exactly?
- If you run the SDE from the same starting noise twice (same seed), will the endpoints be the same?
- Which method produces more diverse samples from the same starting points?

In [ ]:
# Your SDE vs ODE trajectory code goes here.
#
# Suggested structure:
#
# 1. Define a function sample_sde(model, alpha_bars, betas, alphas, x_T, T)
#    that implements DDPM-style reverse sampling with stochastic noise.
#    Return both the final samples AND the full trajectory (list of x at each step).
#
# 2. Define a function sample_ode(model, alpha_bars, x_T, T)
#    that implements DDIM-style deterministic sampling (no stochastic noise).
#    Return both the final samples AND the full trajectory.
#
# 3. Generate N starting points from pure noise (x_T ~ N(0, I)).
#
# 4. Run both methods from the SAME starting points.
#    For the SDE, run it TWICE to show different endpoints.
#    For the ODE, run it TWICE to show identical endpoints.
#
# 5. Plot:
#    - Panel 1: SDE trajectories (wiggly, different colors per trajectory)
#    - Panel 2: ODE trajectories (smooth, different colors per trajectory)
#    - Panel 3: SDE run 1 vs run 2 endpoints (should differ)
#    - Panel 4: ODE run 1 vs run 2 endpoints (should match)
#    - Mark the data cluster centers for reference.
#
# Remember:
# - The model, alpha_bars, betas, alphas are already defined from Exercise 3.
# - model.eval() is already set.
# - Use torch.no_grad() for inference.
# - For the SDE last step (t=0), set the noise z = 0.
# - For the ODE, clip predicted_x0 to a reasonable range (e.g., -5 to 5)
#   to prevent numerical instability at early timesteps.



<details>
<summary>💡 Solution</summary>

The core insight is that the SDE and ODE are two ways to traverse the same generative landscape. The SDE adds exploration (random noise at each step), making paths wiggly and diverse. The ODE commits to a deterministic path, making it smooth and repeatable. This is the same distinction as DDPM vs DDIM—now seen through the formal SDE/ODE lens.

```python
@torch.no_grad()
def sample_sde(model, alpha_bars, betas, alphas, x_T, T):
    """Reverse SDE sampling (DDPM-like): deterministic denoising + stochastic noise."""
    trajectory = [x_T.clone()]
    x = x_T.clone()
    
    for t_val in reversed(range(T)):
        t_tensor = torch.full((x.shape[0],), t_val, dtype=torch.long)
        eps_pred = model(x, t_tensor)
        
        alpha_t = alphas[t_val]
        beta_t = betas[t_val]
        ab_t = alpha_bars[t_val]
        
        # DDPM reverse step: remove predicted noise, add fresh stochastic noise
        mean = (1 / torch.sqrt(alpha_t)) * (x - (beta_t / torch.sqrt(1 - ab_t)) * eps_pred)
        
        # Stochastic noise (zero at last step)
        if t_val > 0:
            sigma_t = torch.sqrt(beta_t)
            z = torch.randn_like(x)
            x = mean + sigma_t * z
        else:
            x = mean
        
        if t_val % 5 == 0:  # Store every 5th step for trajectory plotting
            trajectory.append(x.clone())
    
    return x, trajectory


@torch.no_grad()
def sample_ode(model, alpha_bars, x_T, T):
    """Probability flow ODE sampling (DDIM-like): fully deterministic."""
    trajectory = [x_T.clone()]
    x = x_T.clone()
    
    for t_val in reversed(range(T)):
        t_tensor = torch.full((x.shape[0],), t_val, dtype=torch.long)
        eps_pred = model(x, t_tensor)
        
        ab_t = alpha_bars[t_val]
        ab_prev = alpha_bars[t_val - 1] if t_val > 0 else torch.tensor(1.0)
        
        # DDIM step: predict x_0, then jump to x_{t-1}
        pred_x0 = (x - torch.sqrt(1 - ab_t) * eps_pred) / torch.sqrt(ab_t)
        pred_x0 = pred_x0.clamp(-5, 5)  # Numerical stability
        
        # Deterministic step toward predicted x_0 (no noise!)
        x = torch.sqrt(ab_prev) * pred_x0 + torch.sqrt(1 - ab_prev) * eps_pred
        
        if t_val % 5 == 0:
            trajectory.append(x.clone())
    
    # Final x is the predicted clean sample
    return pred_x0, trajectory


# Generate starting points
n_trajectories = 8
torch.manual_seed(42)
x_T = torch.randn(n_trajectories, 2) * 2  # Starting noise

# Run SDE twice (different endpoints expected)
torch.manual_seed(100)
sde_final_1, sde_traj_1 = sample_sde(model, alpha_bars, betas, alphas, x_T.clone(), T)
torch.manual_seed(200)  # Different seed for stochastic noise
sde_final_2, sde_traj_2 = sample_sde(model, alpha_bars, betas, alphas, x_T.clone(), T)

# Run ODE twice (identical endpoints expected)
ode_final_1, ode_traj_1 = sample_ode(model, alpha_bars, x_T.clone(), T)
ode_final_2, ode_traj_2 = sample_ode(model, alpha_bars, x_T.clone(), T)

# Plot
colors = plt.cm.tab10(np.linspace(0, 1, n_trajectories))
centers_np = np.array([[-2, -2], [-2, 2], [2, -2], [2, 2]])

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Panel 1: SDE trajectories (run 1)
ax = axes[0]
for i in range(n_trajectories):
    pts = np.array([t[i].numpy() for t in sde_traj_1])
    ax.plot(pts[:, 0], pts[:, 1], '-', color=colors[i], alpha=0.6, linewidth=1)
    ax.plot(pts[0, 0], pts[0, 1], 'o', color=colors[i], markersize=6)
    ax.plot(pts[-1, 0], pts[-1, 1], 's', color=colors[i], markersize=6)
for c in centers_np:
    ax.plot(c[0], c[1], '*', color='#f97316', markersize=15, markeredgecolor='white')
ax.set_title('SDE Trajectories\n(wiggly, stochastic)', fontsize=11)
ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_aspect('equal')

# Panel 2: ODE trajectories
ax = axes[1]
for i in range(n_trajectories):
    pts = np.array([t[i].numpy() for t in ode_traj_1])
    ax.plot(pts[:, 0], pts[:, 1], '-', color=colors[i], alpha=0.6, linewidth=1)
    ax.plot(pts[0, 0], pts[0, 1], 'o', color=colors[i], markersize=6)
    ax.plot(pts[-1, 0], pts[-1, 1], 's', color=colors[i], markersize=6)
for c in centers_np:
    ax.plot(c[0], c[1], '*', color='#f97316', markersize=15, markeredgecolor='white')
ax.set_title('ODE Trajectories\n(smooth, deterministic)', fontsize=11)
ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_aspect('equal')

# Panel 3: SDE run 1 vs run 2
ax = axes[2]
s1 = sde_final_1.numpy()
s2 = sde_final_2.numpy()
ax.scatter(s1[:, 0], s1[:, 1], s=60, c='#a78bfa', marker='o', label='SDE run 1', zorder=3)
ax.scatter(s2[:, 0], s2[:, 1], s=60, c='#f97316', marker='^', label='SDE run 2', zorder=3)
for i in range(n_trajectories):
    ax.plot([s1[i, 0], s2[i, 0]], [s1[i, 1], s2[i, 1]], '--', color='white', alpha=0.3)
for c in centers_np:
    ax.plot(c[0], c[1], '*', color='#f97316', markersize=15, markeredgecolor='white')
ax.set_title('SDE: Same Start, Different Ends\n(stochastic = diverse)', fontsize=11)
ax.legend(fontsize=8)
ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_aspect('equal')

# Panel 4: ODE run 1 vs run 2
ax = axes[3]
o1 = ode_final_1.numpy()
o2 = ode_final_2.numpy()
ax.scatter(o1[:, 0], o1[:, 1], s=60, c='#a78bfa', marker='o', label='ODE run 1', zorder=3)
ax.scatter(o2[:, 0], o2[:, 1], s=60, c='#f97316', marker='^', label='ODE run 2', zorder=3)
for c in centers_np:
    ax.plot(c[0], c[1], '*', color='#f97316', markersize=15, markeredgecolor='white')
ax.set_title('ODE: Same Start, Same Ends\n(deterministic = repeatable)', fontsize=11)
ax.legend(fontsize=8)
ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_aspect('equal')

plt.suptitle(
    'Reverse SDE (DDPM-like) vs Probability Flow ODE (DDIM-like)\n'
    'Same trained model, same starting noise, different sampling strategies',
    fontsize=13, y=1.05,
)
plt.tight_layout()
plt.show()

# Quantify the difference
ode_diff = torch.norm(ode_final_1 - ode_final_2, dim=1).mean().item()
sde_diff = torch.norm(sde_final_1 - sde_final_2, dim=1).mean().item()
print(f'ODE: average endpoint difference between runs: {ode_diff:.6f} (should be ~0)')
print(f'SDE: average endpoint difference between runs: {sde_diff:.4f} (should be > 0)')
print()
print('The ODE is deterministic: same start -> same end, every time.')
print('The SDE is stochastic: same start -> different ends, every time.')
print()
print('This is DDPM vs DDIM, seen through the SDE/ODE lens.')
print('Same model. Same score function. Different sampling strategy.')
```

**Key decisions:**
- Storing every 5th step keeps trajectories visible without overwhelming the plot.
- Clamping `pred_x0` to [-5, 5] in the ODE prevents numerical instability at early timesteps where $\bar\alpha_t$ is very small.
- Using different random seeds for the two SDE runs ensures different stochastic noise, producing different endpoints.
- The ODE does not use any random noise, so no seed matters after the starting points are fixed.

**Common mistakes:**
- Adding stochastic noise in the ODE sampler. The ODE is deterministic—no `z * sigma` term.
- Forgetting to set `z = 0` for the last SDE step (t=0). Adding noise at the very end corrupts the final sample.
- Not using `torch.no_grad()`. This wastes memory by storing gradients during inference.

</details>

---

## Key Takeaways

1. **The score function is the gradient of log probability.** $\text{score}(x) = \nabla_x \log p(x)$. At every point in data space, it gives you a direction toward higher probability. It is zero at peaks (nowhere better to go) and large far from peaks (strong pull toward data). It is a compass toward likely data.

2. **The score field simplifies with noise.** Adding noise to the data distribution smooths the score field. At high noise, the field is nearly linear (easy for a model to learn). At low noise, the field is complex (hard to learn). This is why the noise schedule matters—it controls the difficulty progression from easy tasks to hard tasks.

3. **DDPM's noise prediction IS the score function (up to scaling).** $\text{score}(x_t, t) \approx -\epsilon_\theta(x_t, t) / \sqrt{1 - \bar\alpha_t}$. The model was always learning to point toward data. Nothing new was trained—the score was hiding inside DDPM all along.

4. **The reverse SDE (DDPM-like) and probability flow ODE (DDIM-like) are two ways to sample.** Both use the same trained model and the same score function. The SDE adds stochastic exploration—wiggly paths, diverse outputs. The ODE commits deterministically—smooth paths, repeatable outputs. Same landscape, different traversal strategy.

5. **This is the unifying framework.** DDPM, DDIM, DPM-Solver—different vocabulary for the same underlying process. A trained noise prediction network is a score estimator. Samplers are different strategies for following the score.